In [28]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score
from sklearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler

# Load the dataset
df = pd.read_csv('merged_df.csv')

# Display the first few rows of the dataframe and its summary information
df_info = df.info()
df_head = df.head()

df_info, df_head

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10861 entries, 0 to 10860
Data columns (total 45 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   date                    10861 non-null  object 
 1   humidity                10861 non-null  float64
 2   temperature             10861 non-null  float64
 3   pressure                10861 non-null  float64
 4   pm2.5_alt               10861 non-null  float64
 5   scattering_coefficient  10861 non-null  float64
 6   visual_range            10861 non-null  float64
 7   0.3_um_count            10861 non-null  float64
 8   5.0_um_count            10861 non-null  float64
 9   pm1.0_atm               10861 non-null  float64
 10  pm2.5_atm               10861 non-null  float64
 11  pm2.5_cf_1              10861 non-null  float64
 12  pm10.0_atm              10861 non-null  float64
 13  sensor_id               10861 non-null  int64  
 14  last_modified           10861 non-null

(None,
          date  humidity  temperature  pressure  pm2.5_alt  \
 0  2019-10-11    15.918       68.038   926.446        3.3   
 1  2019-08-27    10.035       90.543   923.192        0.4   
 2  2019-12-13    55.267       60.675   933.648        0.9   
 3  2019-01-23    32.291       54.944   937.922        0.6   
 4  2019-02-15    70.416       45.442   922.313        0.5   
 
    scattering_coefficient  visual_range  0.3_um_count  5.0_um_count  \
 0                    11.4         164.5      761.3590        0.8265   
 1                     1.6         327.9      105.2000        0.0625   
 2                     3.0         286.5      200.6605        0.5825   
 3                     2.3         306.6      151.0885        0.3530   
 4                     1.7         324.8      111.5450        0.6515   
 
    pm1.0_atm  ...  wspd  wpgt    pres  tsun unique_incident_count  \
 0     3.6135  ...   5.4   NaN  1015.2   NaN                   0.0   
 1     0.1225  ...  10.1   NaN  1010.0   NaN 

In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

# Load the dataset
df = pd.read_csv('merged_df.csv')  # Adjust path as necessary

# Feature Engineering: Extract year, month, and day from the 'date' column
df['year'] = pd.to_datetime(df['date']).dt.year
df['month'] = pd.to_datetime(df['date']).dt.month
df['day'] = pd.to_datetime(df['date']).dt.day

# Create dummy variables for year, month, and day
year_dummies = pd.get_dummies(df['year'], prefix='year').astype(int)
month_dummies = pd.get_dummies(df['month'], prefix='month').astype(int)
day_dummies = pd.get_dummies(df['day'], prefix='day').astype(int)

# Concatenate the original dataframe df with the new dummy variable dataframes
df = pd.concat([df, year_dummies, month_dummies, day_dummies], axis=1)

# Drop the original 'date' column and any other non-numeric or unnecessary columns
df.drop(['date'], axis=1, inplace=True)  # Add or remove columns as needed

In [30]:
df

,humidity,temperature,pressure,pm2.5_alt,scattering_coefficient,visual_range,0.3_um_count,5.0_um_count,pm1.0_atm,pm2.5_atm,...,day_22,day_23,day_24,day_25,day_26,day_27,day_28,day_29,day_30,day_31
0,15.918,68.038,926.446,3.3,11.4,164.5,761.3590,0.8265,3.6135,5.2745,...,0,0,0,0,0,0,0,0,0,0
1,10.035,90.543,923.192,0.4,1.6,327.9,105.2000,0.0625,0.1225,0.2015,...,0,0,0,0,0,1,0,0,0,0
2,55.267,60.675,933.648,0.9,3.0,286.5,200.6605,0.5825,0.4445,0.9785,...,0,0,0,0,0,0,0,0,0,0
3,32.291,54.944,937.922,0.6,2.3,306.6,151.0885,0.3530,0.4850,0.6940,...,0,1,0,0,0,0,0,0,0,0
4,70.416,45.442,922.313,0.5,1.7,324.8,111.5450,0.6515,0.1815,0.3890,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10856,24.791,85.291,981.358,4.6,14.4,143.2,957.1125,0.7940,4.3025,6.7805,...,0,0,0,0,0,0,0,0,0,0
10857,28.001,86.987,981.957,8.7,30.8,83.0,2054.1660,0.8565,10.4465,14.1310,...,0,0,0,0,0,0,0,0,0,0
10858,47.927,54.770,988.388,19.2,50.3,55.4,3355.1960,4.0210,17.4880,29.1230,...,0,0,0,0,0,0,1,0,0,0
10859,27.300,87.433,983.447,6.9,22.3,106.2,1484.4430,1.0270,7.2770,10.7575,...,0,0,0,0,0,0,0,0,0,0


In [37]:
# Basic data preprocessing
df = df.select_dtypes(include=[np.number]).dropna(axis=1, thresh=int(0.8 * len(df)))

# Impute missing values
imputer = SimpleImputer(strategy='mean')
data_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

# Feature and target separation
X = data_imputed.drop('target', axis=1)
y = data_imputed['target']

In [38]:
has_nan = X.isnull().values.any()

if has_nan:
    print("X has NaN values.")
else:
    print("X does not have any NaN values.")

X does not have any NaN values.


SMOTE Upsampling on Minority Class

In [57]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import numpy as np

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Apply SMOTE to balance the dataset with only a portion of the minority class upsampled
smote = SMOTE(sampling_strategy=0.5, random_state=42)  # Upsample minority class to 20% of the majority class
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Add noise to SMOTE-sampled data
noise_factor = 0.1  # Adjust as needed for more or less noise
X_train_smote_noisy = X_train_smote + noise_factor * np.random.normal(loc=0, scale=1, size=X_train_smote.shape)

# Since Logistic Regression benefits from scaling, we scale the features
scaler = StandardScaler()
X_train_smote_noisy_scaled = scaler.fit_transform(X_train_smote_noisy)
X_test_scaled = scaler.transform(X_test)

# Initialize models
log_reg = LogisticRegression(random_state=42, max_iter=100)
random_forest = RandomForestClassifier(random_state=42)

# Train models
log_reg.fit(X_train_smote_noisy_scaled, y_train_smote)
random_forest.fit(X_train_smote_noisy_scaled, y_train_smote)

# Evaluate models
log_reg_score = log_reg.score(X_test_scaled, y_test)
random_forest_score = random_forest.score(X_test_scaled, y_test)

print("Logistic Regression Test Accuracy:", log_reg_score)
print("Random Forest Test Accuracy:", random_forest_score)


Logistic Regression Test Accuracy: 1.0
Random Forest Test Accuracy: 1.0


In [58]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import numpy as np

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply SMOTE to balance the minority class
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Apply RandomUnderSampler to downsample the majority class
rus = RandomUnderSampler(random_state=42)
X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)

# Combine the resampled minority class and the downsampled majority class
X_train_combined = np.vstack((X_train_smote, X_train_resampled))
y_train_combined = np.concatenate((y_train_smote, y_train_resampled))

# Shuffle the combined data
shuffle_indices = np.random.permutation(len(X_train_combined))
X_train_combined = X_train_combined[shuffle_indices]
y_train_combined = y_train_combined[shuffle_indices]

# Add noise to the combined data
noise_factor = 0.1  # Adjust as needed for more or less noise
X_train_combined_noisy = X_train_combined + noise_factor * np.random.normal(loc=0, scale=1, size=X_train_combined.shape)

# Since Logistic Regression benefits from scaling, we scale the features
scaler = StandardScaler()
X_train_combined_noisy_scaled = scaler.fit_transform(X_train_combined_noisy)
X_test_scaled = scaler.transform(X_test)

# Initialize models
log_reg = LogisticRegression(random_state=42, max_iter=1000)
random_forest = RandomForestClassifier(random_state=42)

# Train models
log_reg.fit(X_train_combined_noisy_scaled, y_train_combined)
random_forest.fit(X_train_combined_noisy_scaled, y_train_combined)

# Evaluate models
log_reg_score = log_reg.score(X_test_scaled, y_test)
random_forest_score = random_forest.score(X_test_scaled, y_test)

print("Logistic Regression Test Accuracy:", log_reg_score)
print("Random Forest Test Accuracy:", random_forest_score)


/Users/wen-chilee/anaconda3/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


Logistic Regression Test Accuracy: 1.0
Random Forest Test Accuracy: 1.0


In [59]:
X_train

,humidity,temperature,pressure,pm2.5_alt,scattering_coefficient,visual_range,0.3_um_count,5.0_um_count,pm1.0_atm,pm2.5_atm,...,day_22,day_23,day_24,day_25,day_26,day_27,day_28,day_29,day_30,day_31
393,18.200,77.465,925.794,0.6,2.6,298.3,170.8965,0.1805,0.0740,0.2485,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10052,39.662,60.053,998.665,33.4,79.8,36.9,5318.2565,8.2895,25.3530,43.5715,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6208,37.803,76.901,959.089,8.0,26.4,93.4,1763.0105,1.5170,9.6460,13.2395,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8575,16.301,80.889,768.830,65.7,155.7,19.8,10376.7855,9.1555,43.6645,77.3740,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8426,34.256,77.576,773.316,3.4,13.9,145.9,929.9005,0.1430,4.4525,5.5610,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,30.822,66.794,966.222,0.9,3.9,266.7,256.7000,0.2650,0.4495,0.7905,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5191,43.733,68.551,956.788,1.1,4.2,259.3,280.1265,0.4750,0.5780,1.0545,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5390,33.000,75.757,962.720,3.6,13.6,147.9,909.1375,0.8205,4.2565,5.6390,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
860,19.521,79.677,955.073,0.7,2.6,297.0,174.0280,0.1725,0.0855,0.3300,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [63]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import numpy as np

# Randomly select 10 features
selected_feature_indices = np.random.choice(X_train.shape[1], size=20, replace=False)
X_train_selected = X_train.iloc[:, selected_feature_indices]
X_test_selected = X_test.iloc[:, selected_feature_indices]

# Apply SMOTE to balance the minority class
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_selected, y_train)

# Apply RandomUnderSampler to downsample the majority class
rus = RandomUnderSampler(random_state=42)
X_train_resampled, y_train_resampled = rus.fit_resample(X_train_selected, y_train)

# Combine the resampled minority class and the downsampled majority class
X_train_combined = np.vstack((X_train_smote, X_train_resampled))
y_train_combined = np.concatenate((y_train_smote, y_train_resampled))

# Shuffle the combined data
shuffle_indices = np.random.permutation(len(X_train_combined))
X_train_combined = X_train_combined[shuffle_indices]
y_train_combined = y_train_combined[shuffle_indices]

# Add noise to the combined data
noise_factor = 0.1  # Adjust as needed for more or less noise
X_train_combined_noisy = X_train_combined + noise_factor * np.random.normal(loc=0, scale=1, size=X_train_combined.shape)

# Since Logistic Regression benefits from scaling, we scale the features
scaler = StandardScaler()
X_train_combined_noisy_scaled = scaler.fit_transform(X_train_combined_noisy)
X_test_scaled = scaler.transform(X_test_selected)

# Initialize models
log_reg = LogisticRegression(random_state=42, max_iter=1000)
random_forest = RandomForestClassifier(random_state=42)

# Train models
log_reg.fit(X_train_combined_noisy_scaled, y_train_combined)
random_forest.fit(X_train_combined_noisy_scaled, y_train_combined)

# Evaluate models
log_reg_score = log_reg.score(X_test_scaled, y_test)
random_forest_score = random_forest.score(X_test_scaled, y_test)

print("Logistic Regression Test Accuracy:", log_reg_score)
print("Random Forest Test Accuracy:", random_forest_score)


/Users/wen-chilee/anaconda3/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


Logistic Regression Test Accuracy: 0.7611596870685688
Random Forest Test Accuracy: 0.9682466635987115


In [40]:
# Logistic Regression: Perform cross-validation
log_reg_scores = cross_val_score(log_reg, X_train_smote_scaled, y_train_smote, cv=5, scoring='accuracy')
print("Logistic Regression - Accuracy for each fold: ", log_reg_scores)
print("Logistic Regression - Mean cross-validation accuracy: ", log_reg_scores.mean())

# Random Forest: Perform cross-validation
# Note: No need to scale features for Random Forest, using non-scaled data
rf_scores = cross_val_score(random_forest, X_train_smote, y_train_smote, cv=5, scoring='accuracy')
print("Random Forest - Accuracy for each fold: ", rf_scores)
print("Random Forest - Mean cross-validation accuracy: ", rf_scores.mean())


Logistic Regression - Accuracy for each fold:  [1. 1. 1. 1. 1.]
Logistic Regression - Mean cross-validation accuracy:  1.0
Random Forest - Accuracy for each fold:  [1. 1. 1. 1. 1.]
Random Forest - Mean cross-validation accuracy:  1.0
